In [1]:
print("hello world")

hello world


In [2]:
2-4

-2

https://huggingface.co/docs/transformers/model_doc/speech_to_text

In [1]:
import torch

In [2]:
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration

/Users/curcuqui/Documents/GitHub/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import load_dataset

/Users/curcuqui/Documents/GitHub/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")

In [5]:
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

In [6]:
ds = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

Found cached dataset librispeech_asr_demo (C:/Users/Usuario/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [7]:
inputs = processor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="pt")

In [8]:
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

C:\Users\Usuario\Anaconda3\lib\site-packages\transformers\generation\utils.py:1357: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [9]:
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [10]:
transcription

['mister quilter is the apostle of the middle classes and we are glad to welcome his gospel']

***

In [16]:
ds[0]["audio"]["sampling_rate"]

16000

In [15]:
ds[0]["audio"]["array"]

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

+ https://pypi.org/project/audio2numpy/
+ https://realpython.com/python-speech-recognition/

In [2]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1 #2
fs = 44100  # Record at 44100 samples per second
seconds = 10
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [13]:
import numpy as np
i = np.arange(28*28).reshape(28, 28)
y = np.frombuffer(frames, dtype=i.dtype)

TypeError: a bytes-like object is required, not 'list'

In [7]:
P = pyaudio.PyAudio()

stream = P.open(rate=fs, format=pyaudio.paInt16, channels=1, output=True)
stream.write(x.tobytes())

stream.close() # this blocks until sound finishes playing

P.terminate()

AttributeError: 'bytes' object has no attribute 'astype'